In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
import threading
import pygame

# Инициализация pygame для воспроизведения звука
pygame.mixer.init()
alarm_sound = "alarm.mp3"  # Убедитесь, что файл alarm.mp3 находится в текущей директории
pygame.mixer.music.load(alarm_sound)

# Загрузка модели
model = YOLO('yolov8n.pt')
model.to('cpu')  # Используем CPU

# Открытие видеопотока
cap = cv2.VideoCapture(0)  # 0 для камеры, или замените на путь к видеофайлу

if not cap.isOpened():
    print("Не удалось открыть видеопоток.")
    exit()

# Убедимся, что класс "person" существует
print(model.names)  # Вывод доступных классов
class_index = list(model.names.values()).index('person')

# Порог для близкого расстояния (в пикселях)
distance_threshold = 300

# Функция для звукового сигнала
def play_alarm():
    if not pygame.mixer.music.get_busy():  # Проверяем, что звук не проигрывается
        pygame.mixer.music.play()

while True:
    ret, frame = cap.read()
    if not ret:
        print("Конец видео или ошибка при чтении кадра.")
        break

    # Обработка кадра
    results = model.predict(source=frame, conf=0.25)

    # Список центров и bounding box'ов для людей
    centers = []
    boxes = []

    for box in results[0].boxes:
        if int(box.cls) == class_index:  # Проверка на класс 'person'
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Координаты bbox
            center_x = (x1 + x2) // 2
            center_y = (y1 + y2) // 2
            centers.append((center_x, center_y))
            boxes.append((x1, y1, x2, y2))

    # Проверка на близость людей
    close_pairs = set()  # Хранит пары индексов близких людей
    for i, center1 in enumerate(centers):
        for j, center2 in enumerate(centers):
            if i != j:  # Не сравниваем человека с самим собой
                distance = np.sqrt((center1[0] - center2[0]) ** 2 + (center1[1] - center2[1]) ** 2)
                if distance < distance_threshold:
                    close_pairs.add(i)
                    close_pairs.add(j)

    # Рисование рамок и звуковое предупреждение
    alarm_triggered = False
    for i, (x1, y1, x2, y2) in enumerate(boxes):
        color = (0, 0, 255) if i in close_pairs else (255, 0, 0)  # Красный, если близко; синий, если нет
        if i in close_pairs:
            alarm_triggered = True
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        label = "person"
        cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Воспроизведение звука при срабатывании тревоги
    if alarm_triggered:
        threading.Thread(target=play_alarm, daemon=True).start()

    # Показ результата
    cv2.imshow("Real-Time Person Detection", frame)

    # Выход при нажатии клавиши 'q'
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
pygame.mixer.quit()


pygame 2.6.1 (SDL 2.28.4, Python 3.12.0)
Hello from the pygame community. https://www.pygame.org/contribute.html
{0: 'person', 1: 'bicycle', 2: 'car', 3: 'motorcycle', 4: 'airplane', 5: 'bus', 6: 'train', 7: 'truck', 8: 'boat', 9: 'traffic light', 10: 'fire hydrant', 11: 'stop sign', 12: 'parking meter', 13: 'bench', 14: 'bird', 15: 'cat', 16: 'dog', 17: 'horse', 18: 'sheep', 19: 'cow', 20: 'elephant', 21: 'bear', 22: 'zebra', 23: 'giraffe', 24: 'backpack', 25: 'umbrella', 26: 'handbag', 27: 'tie', 28: 'suitcase', 29: 'frisbee', 30: 'skis', 31: 'snowboard', 32: 'sports ball', 33: 'kite', 34: 'baseball bat', 35: 'baseball glove', 36: 'skateboard', 37: 'surfboard', 38: 'tennis racket', 39: 'bottle', 40: 'wine glass', 41: 'cup', 42: 'fork', 43: 'knife', 44: 'spoon', 45: 'bowl', 46: 'banana', 47: 'apple', 48: 'sandwich', 49: 'orange', 50: 'broccoli', 51: 'carrot', 52: 'hot dog', 53: 'pizza', 54: 'donut', 55: 'cake', 56: 'chair', 57: 'couch', 58: 'potted plant', 59: 'bed', 60: 'dining table